# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298342155082                   -0.85    5.0
  2   -8.300167919753       -2.74       -1.26    1.1
  3   -8.300445200710       -3.56       -1.89    2.5
  4   -8.300462335916       -4.77       -2.76    2.6
  5   -8.300464163726       -5.74       -3.04   10.5
  6   -8.300464375499       -6.67       -3.19    2.0
  7   -8.300464507954       -6.88       -3.32    1.6
  8   -8.300464573601       -7.18       -3.44    1.1
  9   -8.300464627946       -7.26       -3.66    1.0
 10   -8.300464636228       -8.08       -3.79    1.0
 11   -8.300464642245       -8.22       -4.00    2.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67458014973                   -0.70    6.1
  2   -16.67869280855       -2.39       -1.14    1.0
  3   -16.67922492415       -3.27       -1.87    3.9
  4   -16.67928228106       -4.24       -2.78    3.6
  5   -16.67928611688       -5.42       -3.22    4.9
  6   -16.67928621144       -7.02       -3.51    3.1
  7   -16.67928622111       -8.01       -3.96    1.0
  8   -16.67928622415       -8.52       -4.59    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32548076191                   -0.56    7.1
  2   -33.33274510159       -2.14       -1.00    1.0
  3   -33.33413774011       -2.86       -1.74    5.4
  4   -33.33428105764       -3.84       -2.64    3.8
  5   -33.33694326948       -2.57       -2.52    9.4
  6   -33.33694357528       -6.51       -2.52    5.0
  7   -33.33694121990   +   -5.63       -2.56    3.6
  8   -33.33686710745   +   -4.13       -2.63    1.6
  9   -33.33687542933       -5.08       -2.68    1.0
 10   -33.33688832819       -4.89       -2.73    1.0
 11   -33.33690233922       -4.85       -2.79    1.0
 12   -33.33692428195       -4.66       -2.94    1.6
 13   -33.33693365689       -5.03       -3.10    2.1
 14   -33.33693826025       -5.34       -3.24    2.5
 15   -33.33694259873       -5.36       -3.58    4.2
 16   -33.33694384052       -5.91       -4.25    3.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298313816763                   -0.85    5.2
  2   -8.300232995295       -2.72       -1.58    1.0
  3   -8.300360111622       -3.90       -2.22    4.1
  4   -8.300312563016   +   -4.32       -2.16    7.2
  5   -8.300464207075       -3.82       -3.60    1.4
  6   -8.300464585635       -6.42       -3.88    4.1
  7   -8.300464636159       -7.30       -4.28    1.9


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31924148135                   -0.56    7.0
  2   -33.32257553954       -2.48       -1.28    1.1
  3   -26.39884975302   +    0.84       -0.65    7.8
  4   -33.18543016090        0.83       -1.45    6.1
  5   -33.28974247072       -0.98       -1.42    3.6
  6   -33.27593747069   +   -1.86       -1.64    3.8
  7   -32.66570356116   +   -0.21       -1.15    5.0
  8   -33.33456698749       -0.17       -2.26    5.1
  9   -33.33286866515   +   -2.77       -2.07    2.9
 10   -33.33654111313       -2.44       -2.51    2.6
 11   -33.33667399446       -3.88       -2.80    2.0
 12   -33.33688724262       -3.67       -3.11    2.8
 13   -33.33692783215       -4.39       -3.35    2.9
 14   -33.33693830075       -4.98       -3.55    2.5
 15   -33.33693291024   +   -5.27       -3.53    4.4
 16   -33.33694332226       -4.98       -4.15    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.